In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import requests
import json
import matplotlib.pyplot as plt
from model_training_process import *
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices("GPU")
print(f"tf.config.list_physical_devices('GPU'): {tf.config.list_physical_devices('GPU')}")

tf.config.list_physical_devices('GPU'): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
class_df = pd.read_csv(filepath_or_buffer="..\\tables\\corrected_class_df_pinyin_label_table.csv")
class_df

,bopomofo,pinyin,hu_yin,she2_yin,hu_yin_label,she2_yin_label,class_label
0,ㄓ,zhi,純聲符,翹舌音,0,5,0
1,ㄔ,chi,純聲符,翹舌音,0,5,0
2,ㄕ,shi,純聲符,翹舌音,0,5,0
3,ㄖ,ri,純聲符,翹舌音,0,5,0
4,ㄗ,zi,純聲符,平舌音,0,6,1
...,...,...,...,...,...,...,...
402,ㄒㄩㄣ,xun,撮口呼,舌面音,4,4,20
403,ㄩㄥ,yong,撮口呼,純韻符,4,0,18
404,ㄐㄩㄥ,jiong,撮口呼,舌面音,4,4,20
405,ㄑㄩㄥ,qiong,撮口呼,舌面音,4,4,20


In [2]:
npy_file_path = glob("..\\..\\data\\*EduVer\\*.npy", recursive=True)
print(f"len(npy_file_path) = {len(npy_file_path)}")

len(npy_file_path) = 67173


In [8]:
npy_file_path_selected = list()
for npy_file in npy_file_path:
    pinyin = npy_file.split("_")[1]
    pinyin_non_accent = pinyin[:-1] if pinyin[-1].isdigit() else pinyin
    if pinyin_non_accent in list(class_df["pinyin"]):
        npy_file_path_selected.append(npy_file)
print(f"len(npy_file_path_selected) = {len(npy_file_path_selected)}")

len(npy_file_path_selected) = 66852


檔名音調: ' ', 2, 3, 4
訓練標籤:  1 , 2, 3, 4, 5

In [9]:
data_accent_label = list()
for npy_file in tqdm(npy_file_path):
    pinyin = npy_file.split("_")[1]
    accent_label = int(pinyin[-1] if pinyin[-1].isdigit() else 1)
    data_accent_label.append(accent_label)
data_accent_label = np.array(data_accent_label)
print(f"len(data_accent_label) = {len(data_accent_label)}")

100%|██████████| 67173/67173 [00:00<00:00, 1751442.10it/s]

len(data_class_label) = 67173
